In [1]:
%load_ext autoreload
%autoreload 2

import ncem
import numpy as np
import seaborn as sns
import tensorflow as tf



from scipy.stats import ttest_rel, ttest_ind

sns.set_palette("colorblind")

# paths
data_path_base ="./input-data/raw-data/"
out_path = "./output-data/Hartmann-2021/"
fn_out_cv = out_path + "/results/"

2023-01-24 01:45:24.203404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 01:45:24.300428: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 01:45:24.704363: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 01:45:24.704411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

## If errors occur with CUDA

- If `InternalError: libdevice not found at ./libdevice.10.bc [Op:__inference_one_e_step_2806]`
    - Include in shell $PATH:
        `export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda`


In [2]:
# - if `"Attempting to perform BLAS operation using StreamExecutor without BLAS support`
#   try setting a dedicate amount of GPU vram:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000)]) # 1500 [MB]
    except RuntimeError as e:
        print('ERROR')

2023-01-24 01:45:26.290928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:45:26.296724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:45:26.296853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Dataset specific inputs

In [3]:
data_set = 'hartmann'
data_path = data_path_base + '/Hartmann-2021/'
log_transform = False
use_domain = True
scale_node_size=False
merge_node_types_predefined = True
covar_selection = []
output_layer='linear'


# Manual inputs

In [4]:
model_class = 'interactions'
optimizer = 'adam'
domain_type = 'patient'

learning_rate = 0.05
l1 = 0.
l2 = 0.

batch_size = 58
radius = 10
n_eval_nodes = 10

gs_id = f"tutorial_{model_class}_{radius}_{data_set}_{domain_type}"



# Model and training


In [5]:
ncv = 3
epochs = 2000 if "tutorial" not in gs_id else 10 
epochs_warmup = 0
max_steps_per_epoch = 20
patience = 100
lr_schedule_min_lr = 1e-10
lr_schedule_factor = 0.5
lr_schedule_patience = 50
val_bs = 16
max_val_steps_per_epoch = 10
shuffle_buffer_size = None

feature_space_id = "standard"
cond_feature_space_id = "type"

use_covar_node_label = False
use_covar_node_position = False
use_covar_graph_covar = False

In [6]:
trainer = ncem.train.TrainModelInteractions()
trainer.init_estim(log_transform=log_transform)

trainer.estimator.get_data(
    data_origin=data_set,
    data_path=data_path,
    radius=radius,
    graph_covar_selection=covar_selection,
    node_label_space_id=cond_feature_space_id,
    node_feature_space_id=feature_space_id,
    # feature_transformation=transformation_dict[transform_key],
    use_covar_node_position=use_covar_node_position,
    use_covar_node_label=use_covar_node_label,
    use_covar_graph_covar=use_covar_graph_covar,
    # hold_out_covariate=hold_out_covariate,
    domain_type=domain_type,
    # merge_node_types_predefined=merge_node_types_predefined,
)

/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading data from raw files
registering celldata


/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/ncem/data.py:2389: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  celldata = AnnData(
/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates


100%|██████████| 58/58 [00:00<00:00, 59.08it/s]

Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.
Mean of mean node degree per images across images: 0.000688


In [7]:
trainer.estimator.split_data_node(
    validation_split=0.1,
    test_split=0.1,
    seed=0
)

Using split method: node. 
 Train-test-validation split is based on total number of nodes per patients over all images.

Excluded 0 cells with the following unannotated cell type: [None] 

Whole dataset: 63747 cells out of 58 images from 4 patients.
Test dataset: 6376 cells out of 58 images from 4 patients.
Training dataset: 51930 cells out of 58 images from 4 patients.
Validation dataset: 5738 cells out of 58 images from 4 patients. 



In [8]:
trainer.estimator.init_model(
    optimizer=optimizer,
    learning_rate=learning_rate,
    n_eval_nodes_per_graph=n_eval_nodes,

    l2_coef=l2,
    l1_coef=l1,
    use_interactions=True,
    use_domain=use_domain,
    scale_node_size=scale_node_size,
    output_layer=output_layer,
)
trainer.estimator.model.training_model.summary()

Model: "interaction_linear_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_da_group (InputLayer)    [(None, 4)]          0           []                               
                                                                                                  
 tf.cast (TFOpLambda)           (None, 4)            0           ['input_da_group[0][0]']         
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 4)         0           ['tf.cast[0][0]']                
                                                                                                  
 interaction (InputLayer)       [(None, 10, 64)]     0           []                               
                                                                           

2023-01-24 01:45:31.039383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 01:45:31.041009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:45:31.041402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:45:31.041475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

In [9]:
trainer.estimator.train(
    epochs=epochs,
    epochs_warmup=epochs_warmup,
    batch_size=batch_size,
    max_steps_per_epoch=max_steps_per_epoch,
    validation_batch_size=val_bs,
    max_validation_steps=max_val_steps_per_epoch,
    patience=patience,
    lr_schedule_min_lr=lr_schedule_min_lr,
    lr_schedule_factor=lr_schedule_factor,
    lr_schedule_patience=lr_schedule_patience,
    monitor_partition="val",
    monitor_metric="loss",
    shuffle_buffer_size=shuffle_buffer_size,
    early_stopping=True,
    reduce_lr_plateau=True,
)

Epoch 1/10
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2023-01-24 01:45:32.916848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-24 01:45:32.925249: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f91ac015bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-24 01:45:32.925283: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-01-24 01:45:32.931831: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-24 01:45:33.042872: I tensorflow/compiler/jit/xla_compilation_ca

1/1 - 2s - loss: 37.5784 - custom_mae: 0.4164 - custom_mean_sd: 1.0000 - custom_mse: 0.2498 - custom_mse_scaled: 0.2498 - gaussian_reconstruction_loss: 37.5784 - r_squared: -2.0198e+00 - r_squared_linreg: 3.5917e-04 - val_loss: 34.1795 - val_custom_mae: 0.3031 - val_custom_mean_sd: 0.9512 - val_custom_mse: 0.1457 - val_custom_mse_scaled: 0.1610 - val_gaussian_reconstruction_loss: 34.1795 - val_r_squared: -8.3769e-01 - val_r_squared_linreg: 0.0428 - lr: 0.0500 - 2s/epoch - 2s/step
Epoch 2/10
1/1 - 0s - loss: 34.3184 - custom_mae: 0.3123 - custom_mean_sd: 0.9512 - custom_mse: 0.1526 - custom_mse_scaled: 0.1687 - gaussian_reconstruction_loss: 34.3184 - r_squared: -8.7881e-01 - r_squared_linreg: 0.0449 - val_loss: 31.4373 - val_custom_mae: 0.2314 - val_custom_mean_sd: 0.9048 - val_custom_mse: 0.0890 - val_custom_mse_scaled: 0.1086 - val_gaussian_reconstruction_loss: 31.4373 - val_r_squared: -1.1943e-01 - val_r_squared_linreg: 0.1593 - lr: 0.0500 - 360ms/epoch - 360ms/step
Epoch 3/10
1/1 - 

In [10]:
evaluation_test = trainer.estimator.evaluate_any(
    img_keys=trainer.estimator.img_keys_test,
    node_idx=trainer.estimator.nodes_idx_test
)

In [11]:
split_per_node_type, evaluation_per_node_type = trainer.estimator.evaluate_per_node_type()

Evaluation for CD11c Myeloid with 1199 cells
{'loss': 16.550827026367188, 'custom_mae': 0.13638442754745483, 'custom_mean_sd': 0.6053009629249573, 'custom_mse': 0.03140474483370781, 'custom_mse_scaled': 0.08567404747009277, 'gaussian_reconstruction_loss': 16.550830841064453, 'r_squared': 0.6119348406791687, 'r_squared_linreg': 0.7327884435653687}
Evaluation for CD4 T cells with 14983 cells
{'loss': 16.43077850341797, 'custom_mae': 0.13329735398292542, 'custom_mean_sd': 0.6052987575531006, 'custom_mse': 0.028939399868249893, 'custom_mse_scaled': 0.07900457084178925, 'gaussian_reconstruction_loss': 16.43077850341797, 'r_squared': 0.6307323575019836, 'r_squared_linreg': 0.7424430847167969}
Evaluation for CD68 Myeloid with 3452 cells
{'loss': 16.858522415161133, 'custom_mae': 0.1511072963476181, 'custom_mean_sd': 0.6053009629249573, 'custom_mse': 0.03766746073961258, 'custom_mse_scaled': 0.10276762396097183, 'gaussian_reconstruction_loss': 16.858509063720703, 'r_squared': 0.481691390275955

In [12]:
evaluation_per_node_type['Fibroblast']

{'loss': 16.712860107421875,
 'custom_mae': 0.1410353034734726,
 'custom_mean_sd': 0.6053009629249573,
 'custom_mse': 0.03467809036374092,
 'custom_mse_scaled': 0.0946759432554245,
 'gaussian_reconstruction_loss': 16.712858200073242,
 'r_squared': 0.5345680117607117,
 'r_squared_linreg': 0.6769022345542908}